In [1]:
import random
from collections import defaultdict

In [2]:
supply_chain_logs = [
    "order_received check_inventory inventory_low place_supplier_order receive_material start_production ship_order",
    "order_received check_inventory inventory_ok start_production ship_order",
    "order_received urgent_order check_inventory inventory_low expedite_supplier_order receive_material start_production ship_order",
    "order_received check_inventory inventory_low place_supplier_order receive_material quality_check ship_order"
]


In [3]:
def tokenize_logs(logs):
    sequences = []
    for line in logs:
        tokens = line.lower().split()
        sequences.append(tokens)
    return sequences

sequences = tokenize_logs(supply_chain_logs)
sequences


[['order_received',
  'check_inventory',
  'inventory_low',
  'place_supplier_order',
  'receive_material',
  'start_production',
  'ship_order'],
 ['order_received',
  'check_inventory',
  'inventory_ok',
  'start_production',
  'ship_order'],
 ['order_received',
  'urgent_order',
  'check_inventory',
  'inventory_low',
  'expedite_supplier_order',
  'receive_material',
  'start_production',
  'ship_order'],
 ['order_received',
  'check_inventory',
  'inventory_low',
  'place_supplier_order',
  'receive_material',
  'quality_check',
  'ship_order']]

In [4]:
def build_vocabulary(sequences):
    vocab = sorted(set(token for seq in sequences for token in seq))
    word_to_idx = {w: i for i, w in enumerate(vocab)}
    idx_to_word = {i: w for w, i in word_to_idx.items()}
    return vocab, word_to_idx, idx_to_word

vocab, word_to_idx, idx_to_word = build_vocabulary(sequences)

print("Vocabulary size:", len(vocab))
vocab


Vocabulary size: 11


['check_inventory',
 'expedite_supplier_order',
 'inventory_low',
 'inventory_ok',
 'order_received',
 'place_supplier_order',
 'quality_check',
 'receive_material',
 'ship_order',
 'start_production',
 'urgent_order']

In [5]:
CONTEXT_SIZE = 2  # trigram model

def generate_ngrams(sequences, context_size):
    ngrams = []
    for seq in sequences:
        for i in range(context_size, len(seq)):
            context = tuple(seq[i - context_size:i])
            target = seq[i]
            ngrams.append((context, target))
    return ngrams

ngrams = generate_ngrams(sequences, CONTEXT_SIZE)
ngrams[:5]


[(('order_received', 'check_inventory'), 'inventory_low'),
 (('check_inventory', 'inventory_low'), 'place_supplier_order'),
 (('inventory_low', 'place_supplier_order'), 'receive_material'),
 (('place_supplier_order', 'receive_material'), 'start_production'),
 (('receive_material', 'start_production'), 'ship_order')]

In [6]:
def train_ngram_model(ngrams):
    counts = defaultdict(int)
    context_counts = defaultdict(int)

    for context, target in ngrams:
        counts[(context, target)] += 1
        context_counts[context] += 1

    model = {}
    for (context, target), c in counts.items():
        model[(context, target)] = c / context_counts[context]

    return model

model = train_ngram_model(ngrams)


In [7]:
def generate_sequence(model, start_context, max_steps=10):
    context = tuple(start_context)
    result = list(context)

    for _ in range(max_steps):
        candidates = [(t, p) for ((c, t), p) in model.items() if c == context]
        if not candidates:
            break

        targets, probs = zip(*candidates)
        next_token = random.choices(targets, probs)[0]

        result.append(next_token)
        context = tuple(result[-len(context):])

    return result


In [8]:
start_context = ["order_received", "check_inventory"]
generated_process = generate_sequence(model, start_context)

print("Generated supply chain process:")
print(" → ".join(generated_process))


Generated supply chain process:
order_received → check_inventory → inventory_low → expedite_supplier_order → receive_material → start_production → ship_order
